In [100]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from logging import getLogger

# 常に全てのカラムを表示
pd.options.display.max_columns = None

# データ取り込み

In [101]:
# DAU
dau = pd.read_csv("input/section3-dau.csv", header="infer")
dau = pd.DataFrame(dau)

# DPU
dpu = pd.read_csv("input/section3-dpu.csv", header="infer")
dpu = pd.DataFrame(dpu)

# install履歴
install = pd.read_csv("input/section3-install.csv", header="infer")
install = pd.DataFrame(install)

In [102]:
# データ中身確認
print(dau.head())
print(dpu.head())
print(install.head())

     log_date app_name  user_id
0  2013-06-01  game-01      116
1  2013-06-01  game-01    13491
2  2013-06-01  game-01     7006
3  2013-06-01  game-01      117
4  2013-06-01  game-01    13492
     log_date app_name  user_id  payment
0  2013-06-01  game-01      351     1333
1  2013-06-01  game-01    12796       81
2  2013-06-01  game-01      364      571
3  2013-06-01  game-01    13212      648
4  2013-06-01  game-01    13212     1142
  install_date app_name  user_id
0   2013-04-15  game-01        1
1   2013-04-15  game-01        2
2   2013-04-15  game-01        3
3   2013-04-15  game-01        4
4   2013-04-15  game-01        5


In [103]:
# 結合
dau_dpu = pd.merge(dau, dpu, on=["app_name", "user_id", "log_date"], how="left")
all_df = pd.merge(dau_dpu, install, on=["app_name", "user_id"], how="inner")

In [104]:
dpu.describe()

,user_id,payment
count,884.000000,884.000000
mean,9741.416290,491.864253
std,9214.556223,500.019241
min,3.000000,0.000000
25%,307.000000,81.000000
50%,8745.000000,324.000000
75%,17539.000000,571.000000
max,29154.000000,2571.000000


## データ加工

In [105]:
# 欠損値を埋める
print(all_df.isna().sum())
# 欠損値を0で埋める
all_df = all_df.fillna(0)

print(all_df.isna().sum())

log_date             0
app_name             0
user_id              0
payment         138411
install_date         0
dtype: int64
log_date        0
app_name        0
user_id         0
payment         0
install_date    0
dtype: int64


In [106]:
# 日付修正
log_date_month = all_df['log_date'].str.split('-', expand=True)
log_date_month.columns = ['year', 'month', 'day']

log_date_year_month = log_date_month['year'] + "-" + log_date_month['month']
log_date_year_month.name = "log_date_year_month"
all_df = pd.concat((all_df, log_date_year_month), axis=1)
all_df.head()

install_date_month = all_df['install_date'].str.split('-', expand=True)
install_date_month.columns = ['year', 'month', 'day']

install_date_year_month = install_date_month['year'] + "-" + install_date_month['month']
install_date_year_month.name = "install_date_year_month"
all_df = pd.concat((all_df, install_date_year_month), axis=1)
all_df.head()

,log_date,app_name,user_id,payment,install_date,log_date_year_month,install_date_year_month
0,2013-06-01,game-01,116,0.0,2013-04-17,2013-06,2013-04
1,2013-06-02,game-01,116,0.0,2013-04-17,2013-06,2013-04
2,2013-06-03,game-01,116,0.0,2013-04-17,2013-06,2013-04
3,2013-06-04,game-01,116,0.0,2013-04-17,2013-06,2013-04
4,2013-06-05,game-01,116,0.0,2013-04-17,2013-06,2013-04


In [107]:
# 新しいカラムでフラグ化
all_df['new_commer_flag'] = 0
all_df.loc[all_df['log_date_year_month'] == all_df['install_date_year_month'] , 'new_commer_flag'] = 1
all_df.head()

,log_date,app_name,user_id,payment,install_date,log_date_year_month,install_date_year_month,new_commer_flag
0,2013-06-01,game-01,116,0.0,2013-04-17,2013-06,2013-04,0
1,2013-06-02,game-01,116,0.0,2013-04-17,2013-06,2013-04,0
2,2013-06-03,game-01,116,0.0,2013-04-17,2013-06,2013-04,0
3,2013-06-04,game-01,116,0.0,2013-04-17,2013-06,2013-04,0
4,2013-06-05,game-01,116,0.0,2013-04-17,2013-06,2013-04,0


## 可視化

In [108]:
# EDA
import pandas_profiling as pp
import pixiedust as px
pp.ProfileReport(all_df)


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


Number of variables,8
Number of observations,139295
Total Missing (%),0.0%
Total size in memory,9.6 MiB
Average record size in memory,72.0 B
Numeric,2
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [ ]:
px.display(all_df)

In [110]:
from pivottablejs import pivot_ui
pivot_ui(all_df)

In [111]:
# 月次集計
mau = all_df.groupby(['log_date_year_month', 'user_id', 'install_date_year_month'], as_index=False).sum()
mau.head()

# user = all_df.groupby(['log_date_year_month', 'user_id', 'install_date_year_month'], as_index=False).count()
# user.head()

# mau = pd.concat((mau, payment), axis=1)


,log_date_year_month,user_id,install_date_year_month,payment,new_commer_flag
0,2013-06,1,2013-04,0.0,0
1,2013-06,2,2013-04,0.0,0
2,2013-06,3,2013-04,14994.0,0
3,2013-06,4,2013-04,0.0,0
4,2013-06,6,2013-04,0.0,0


In [112]:
payment = mau['payment'] .round(-3)
payment.name = "payment_round"
print(payment)
mau = pd.concat((mau, payment), axis=1)
mau_month = mau.groupby(['log_date_year_month', 'payment_round'], as_index=False).count()
mau_month

0            0.0
1            0.0
2        15000.0
3            0.0
4            0.0
5            0.0
6            0.0
7            0.0
8         2000.0
9            0.0
10           0.0
11           0.0
12        3000.0
13           0.0
14           0.0
15           0.0
16           0.0
17           0.0
18        9000.0
19        3000.0
20           0.0
21           0.0
22           0.0
23           0.0
24           0.0
25           0.0
26           0.0
27           0.0
28           0.0
29           0.0
          ...   
27006        0.0
27007        0.0
27008        0.0
27009        0.0
27010        0.0
27011        0.0
27012        0.0
27013        0.0
27014        0.0
27015        0.0
27016        0.0
27017        0.0
27018        0.0
27019        0.0
27020        0.0
27021        0.0
27022        0.0
27023        0.0
27024        0.0
27025        0.0
27026        0.0
27027        0.0
27028        0.0
27029        0.0
27030        0.0
27031        0.0
27032        0.0
27033        0

,log_date_year_month,payment_round,user_id,install_date_year_month,payment,new_commer_flag
0,2013-06,0.0,14743,14743,14743,14743
1,2013-06,1000.0,29,29,29,29
2,2013-06,2000.0,13,13,13,13
3,2013-06,3000.0,6,6,6,6
4,2013-06,4000.0,3,3,3,3
5,2013-06,5000.0,2,2,2,2
6,2013-06,6000.0,1,1,1,1
7,2013-06,7000.0,1,1,1,1
8,2013-06,9000.0,3,3,3,3
9,2013-06,10000.0,1,1,1,1


In [113]:
pivot_ui(mau_month)
# px.display(mau_month)